In [24]:
import cv2 
import math
import numpy as np
import imutils
import time
from numba import jit, cuda
import math as m
from IPython.display import clear_output
def detect_contour(c):
    mxl=0
    mxr=0
    mnl=10**5
    mnr=10**5
    for i in c:
        if(i[0][0]<mnl):
            mnl=i[0][0]
            mnlc=i[0][1]
        if(i[0][0]>mxl):
            mxl=i[0][0]
            mxlc=i[0][1]
        if(i[0][1]<mnr):
            mnr=i[0][1]
            mnrc=i[0][0]
        if(i[0][1]>mxr):
            mxr=i[0][1]
            mxrc=i[0][0]
    return ((mxr-mnr)*(mxl-mnl),((mxrc,mxr),(mnl,mnlc),(mnrc,mnr),(mxl,mxlc)))
def contour(img):
    cnts = imutils.grab_contours(cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE))
    mxa=-1
    x,y = img.shape[:2]
    external_contours = np.zeros((x,y))
    for c in cnts:
        M = cv2.moments(c)
        if M["m00"] != 0:
            cv2.drawContours(external_contours, [c], -1, (255, 255, 255), 1)
            ar,crd = detect_contour(c)
            if(ar>=mxa):
                ct=c
                mxa=ar
                crdf=crd
    external_contours_c = np.zeros((x,y))
    if(mxa!=-1):
        cv2.drawContours(external_contours_c, [ct], -1, (255, 255, 255), 1)
        return(external_contours,external_contours_c,crdf)
    else:
        crdf=0
        return(external_contours,external_contours_c,crdf)
def rotate(image,a,s=1.1):
    if(len(image.shape)<3):
        img1=image
        x,y=img1.shape
        X,Y=int(2.5*x),int(2.5*y)
        image = np.full((X,Y), 0, dtype=np.uint8)
        x0=int((X/2)-(img1.shape[0]/2))
        x1=int((X/2)+(img1.shape[0]/2))
        y0=int((Y/2)-(img1.shape[1]/2))
        y1=int((Y/2)+(img1.shape[1]/2))
        image[x0:x1,y0:y1]=img1
        height, width = image.shape[:2]
        center = (width/2, height/2)
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=s)
        return cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
    elif(len(image.shape)==3):
        img1=image
        x,y,z=img1.shape
        X,Y=int(2*x),int(2*y)
        image = np.full((X,Y,z), 0, dtype=np.uint8)
        x0=int((X/2)-(img1.shape[0]/2))
        x1=int((X/2)+(img1.shape[0]/2))
        y0=int((Y/2)-(img1.shape[1]/2))
        y1=int((Y/2)+(img1.shape[1]/2))
        image[x0:x1,y0:y1,:]=img1
        height, width = image.shape[:2]
        center = (width/2, height/2)
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=s)
        return cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
@jit(target_backend='cuda')
def crop(img):
    mx_up=img.shape[0]
    mx_lt=img.shape[1]
    mx_rt=0
    mx_dn=0
    for j in range (img.shape[0]):
        f=0
        for k in range (img.shape[1]):
            if(img[j,k]!=0):
                img[j,k]=255
                f+=1
                if (f==1 and k<mx_lt):
                    mx_lt=k
                if (f>=1 and k>mx_rt):
                    mx_rt=k
    for j in range (img.shape[1]):
        f=0
        for k in range (img.shape[0]):
            if(img[k,j]!=0):
                img[k,j]=255
                f+=1
                if (f==1 and k<mx_up):
                    mx_up=k
                if (f>=1 and k>mx_dn):
                    mx_dn=k
    mx_lt-=1#9
    mx_dn+=2#10
    mx_rt+=2#10
    mx_up-=1#9
    return (mx_up,mx_dn,mx_lt,mx_rt)
def mask_neg(img):
    image=img#cv2.blur(img, (1,1))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (0, 0, 0), (150, 100, 130))
    target = cv2.bitwise_and(img,img, mask=mask)
    return target,mask
def find_angle(a,b):
    return (m.atan2(a[1] - b[1], a[0] - b[0]) * 180 / m.pi)-90
def distance(p1,p2):
    return (m.sqrt((p1[0] - p2[0])**2+(p1[1] - p2[1])**2))
cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)
while True:
    ret, img_o = cap.read()
    if img_o is not None:
        img,thresh = mask_neg(img_o)
        im,lim,p=contour(thresh)
        if p!=0:
            d1=distance(p[0],p[1])
            d2=distance(p[1],p[2])
            d3=distance(p[0],p[2])
            ta=[d1,d2,d3]
            ta.sort()
            if(d2==ta[1]):
                ang=find_angle(p[1],p[2])
                cv2.circle(lim, p[2], 3, 255, 3)
                cv2.circle(lim, p[1], 3, 255, 3)
            elif(d1==ta[1]):
                ang=find_angle(p[1],p[0])
                cv2.circle(lim, p[0], 3, 255, 3)
                cv2.circle(lim, p[1], 3, 255, 3)
            elif(d3==ta[1]):
                ang=find_angle(p[2],p[0])
                cv2.circle(lim, p[2], 3, 255, 3)
                cv2.circle(lim, p[2], 3, 255, 3)
            mxa=10**9
            an=-6
            for i in range(-5,5,1):
                rimt = rotate(lim,ang+i)
                (mx_up,mx_dn,mx_lt,mx_rt) = crop(rimt)
                cimt=rimt[mx_up:mx_dn,mx_lt:mx_rt]
                if(cimt.shape[0]*cimt.shape[1]<mxa):
                    mxa=cimt.shape[0]*cimt.shape[1]
                    an = i
            rim = rotate(lim,ang+an)
            (mx_up,mx_dn,mx_lt,mx_rt) = crop(rim)
            rimt = rotate(im,ang+an)
            cim=rimt[mx_up:mx_dn,mx_lt:mx_rt]
        cv2.namedWindow("original", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("original", 330, 255)
        cv2.namedWindow("processed1", cv2.WINDOW_NORMAL)        
        cv2.resizeWindow("processed1", 330, 255)
        cv2.namedWindow("processed2", cv2.WINDOW_NORMAL)        
        cv2.resizeWindow("processed2", 330, 255)
        cv2.imshow('original', img_o)
        cv2.imshow('processed1', im)
        cv2.imshow('processed2', cim)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    clear_output(wait=True)
cap.release()
cv2.destroyAllWindows()